# Terraform
Things change over time

In [ ]:
#| default_exp terraform

In [ ]:
#| export
import numpy as np
import sys
import os
import math
import random

#data
from collections import namedtuple
from dataclasses import dataclass,  field, asdict
import json
from typing import List
from enum import Enum

#Jeremy
from dialoghelper import * 
from fastcore.basics import patch
from fasthtml.common import *
from fasthtml.jupyter import *
import httpx

#custom
import inspect
import copy

In [ ]:
from HexMagic.primitives import MapPath, MapSize, MapRect, MapCord 
from HexMagic.primitives import HexGrid, HexPosition ,  HexRegion, GosperCurve
from HexMagic.terrain import Terrain
from HexMagic.terrainpatterns import TerrainPatterns
from HexMagic.climate import TerrainFactory
from HexMagic.hydrology import DrainageBasins

## Some configs

In [ ]:
#| export
@dataclass
class SeismicEvent:
    kind: str  # e.g., "volcano", "erosion", "glacier", "meteor"
    name: str
    order_id: int
    properties: dict  # string key-value pairs
    adjustment: np.ndarray
    caller: str
    
    def encode(self) -> str:
        """Encode the seismic event to a string format."""
        ret = f"kind:{self.kind}\n"
        ret += f"name:{self.name}\n"
        ret += f"caller:{self.caller}\n"
        ret += f"order_id:{self.order_id}\n"
        
        # Encode properties as tab-separated key=value pairs
        prop_str = '\t'.join([f"{k}={v}" for k, v in self.properties.items()])
        ret += f"properties:{prop_str}\n"
        
        ret += "+adjustment:\n"
        # Encode the numpy array as tab-separated values
        ret += '\t'.join([f"{int(x)}" for x in self.adjustment]) + "\n"
        ret += "-adjustment:\n"
        
        return ret
    
    @staticmethod
    def decode(s: str) -> 'SeismicEvent':
        """Decode a string to create a SeismicEvent."""
        lines = s.strip().split('\n')
        
        kind = None
        name = None
        caller = None
        order_id = None
        properties = {}
        adjustment = None
        in_adjustment = False
        
        for line in lines:
            if in_adjustment:
                if line.startswith('-adjustment:'):
                    in_adjustment = False
                else:
                    # Parse the adjustment array
                    adjustment = np.array([float(x) for x in line.split('\t')])
            elif ':' in line:
                key, val = line.split(':', 1)
                key = key.strip()
                val = val.strip()
                
                if key == 'caller':
                    caller = val
                if key == 'kind':
                    kind = val
                elif key == 'name':
                    name = val
                elif key == 'order_id':
                    order_id = int(val)
                elif key == 'properties':
                    # Parse key=value pairs
                    if val:
                        pairs = val.split('\t')
                        for pair in pairs:
                            if '=' in pair:
                                k, v = pair.split('=', 1)
                                properties[k.strip()] = v.strip()
                elif key == '+adjustment':
                    in_adjustment = True
        
        return SeismicEvent(
            kind=kind,
            name=name,
            order_id=order_id,
            properties=properties,
            adjustment=adjustment,
            caller = caller
        )

In [ ]:
#| export
@dataclass
class ClimateRenderConfig:
    mode: str = "climate_zones"
    
    # Rivers
    show_rivers: bool = False
    river_max_count: int = 6
    river_branches: int = 2  # simplify() parameter
    river_color: str = "#1565c0"
    river_opacity: float = 0.7
    river_min_width: float = 1.0
    river_max_width: float = 8.0
    
    # Overlays
    show_elevation_borders: bool = True
    #show_coastline: bool = False
    show_legend: bool = True
    
    debug: bool = False

In [ ]:
#| export
@dataclass
class MapRenderConfig:
    show_hexes: bool = True
    show_coastline: bool = False
    animate: bool = False
    show_biomes: bool = False
    title: str = ""
    subtitle: str = ""
    description:str = ""
    icon:str = ""
    background_color: str = "#81b1e1ff"
    climate: ClimateRenderConfig = None
    shrink:int = 1  # was 'skrink'
    compass:str = None

## Terraform

In [ ]:
#| export
class Terraform:
    def __init__(self, terrain: Terrain,config:MapRenderConfig=None):
        self.terrain = terrain
        self.events: List[SeismicEvent] = []
        # rivers placeholder for later
        self.plates: List[Plate] = []
        if config is None:
            rCofig = ClimateRenderConfig()
            self.config = MapRenderConfig(climate=rCofig)
        else:
            self.config = config

    @property
    def grid(self):
        return self.terrain.hexGrid

    @property
    def builder(self):
        return self.grid.builder



    @staticmethod
    def decode(s: str) -> 'Terraform':
        """Decode a string to create a Terraform."""
        lines = s.split('\n')
        
        terrain_lines = []
        event_lines = []
        config_lines = []
        current_section = None
        current_event = []
        plates = []
        
        i = 0
        while i < len(lines):
            line = lines[i]
            
            if line.startswith('+terrain:'):
                current_section = 'terrain'
            elif line.startswith('-terrain:'):
                current_section = None
            elif line.startswith('+configuration:'):
                current_section = 'configuration'
            elif line.startswith('-configuration:'):
                current_section = None
            elif line.startswith('+plate:'):
                current_section = 'plate'
            elif line.startswith('-plate:'):
                current_section = None
            elif line.startswith('+event:'):
                current_section = 'event'
                current_event = []
            elif line.startswith('-event:'):
                if current_event:
                    event_lines.append('\n'.join(current_event))
                current_event = []
                current_section = None
            elif current_section == 'terrain':
                terrain_lines.append(line)
            elif current_section == 'configuration':
                config_lines.append(line)
            elif current_section == 'event':
                current_event.append(line)
            elif current_section == 'plate':
                plates.append(line)
            
            i += 1
        
        # Decode terrain
        terrain_str = '\n'.join(terrain_lines)
        terrain = Terrain.decode(terrain_str)
        
        # Decode configuration
        config = None
        if config_lines:
            config_dict = json.loads('\n'.join(config_lines))
            # Handle nested ClimateRenderConfig
            climate_dict = config_dict.pop('climate', None)
            climate_config = ClimateRenderConfig(**climate_dict) if climate_dict else None
            config = MapRenderConfig(**config_dict, climate=climate_config)
        
        # Create Terraform
        terraform = Terraform(terrain, config=config)
        
        # Decode events
        for event_str in event_lines:
            event = SeismicEvent.decode(event_str)
            terraform.events.append(event)
        
        terraform.plates = [Plate.decode(x, terrain.hexGrid) for x in plates]
        return terraform

        
        
        # Create Terraform
        terraform = Terraform(terrain)

        
        # Decode events
        for event_str in event_lines:
            event = SeismicEvent.decode(event_str)
            terraform.events.append(event)
        
        terraform.plates = [Plate.decode(x,terrain.hexGrid) for x in plates]
        return terraform

In [ ]:
#| export
@patch
def encode(self: Terraform) -> str:
    """Encode the Terraform to a string format."""
    ret = "=== TERRAFORM ===\n"
    ret += "+terrain:\n"
    ret += self.terrain.encode()
    ret += "-terrain:\n"
    
    # Configuration section
    ret += "+configuration:\n"
    config_dict = asdict(self.config)
    ret += json.dumps(config_dict, indent=2) + "\n"
    ret += "-configuration:\n"
    
    ret += f"event_count:{len(self.events)}\n"

    if len(self.plates) > 0:
        ret += "+plate:\n"
        for plate in self.plates:
            ret += plate.encode()
        ret += "-plate:\n"
    
    for event in self.events:
        ret += "+event:\n"
        ret += event.encode()
        ret += "-event:\n"
    
    return ret